## デコレーター
### 関数やクラスの前後に、任意の処理を追加できる。

In [1]:
# デコレート対象の関数fの呼び出し前後でログを出力するデコレータ
def deco1(f):
    print('deco1 called')
    def wrapper():
        print('before exec')
        # 元の関数が実行
        v = f()
        print('after exec')
        return v
    return wrapper

In [2]:
@deco1
def func():
    print('exec')
    return 1

func()

deco1 called
before exec
exec
after exec


1

元の関数を引数なしで呼び出すことしか想定してないので、デコレート対象の関数が引数を必要都する場合は、以下のようにエラーが出る。

In [3]:
@deco1
def func(x, y):
    print('exec')
    return x, y

func(1, 2)

deco1 called


TypeError: wrapper() takes 0 positional arguments but 2 were given

実際に呼び出される関数は、wrapper()だから、wrapper()が任意の引数を受け取り、元の関数を呼び出す際に受け取った引数をそのまま渡してあげる。

In [4]:
def deco2(f):
    def wrapper(*args, **kwargs):
        print('before exec')
        # 引数を渡して元の関数を実行
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

func(1, 2)を呼び出すと、実際はwrapper(1, 2)が実行される。

In [5]:
@deco2
def func(x,y):
    print('exec')
    return x, y

func(1, 2)

before exec
exec
after exec


(1, 2)

次に、デコレータ自身が引数を受け取るデコレータを実装

In [11]:
def deco3(z):
    # deco2()と同様
    def _deco3(f):
        def wrapper(*args, **kwargs):
            # ここでzを参照できる
            print('before exec', z)
            v = f(*args, **kwargs)
            print('after exec', z)
            return v
        return wrapper
    # デコレータを返す
    return _deco3

In [14]:
@deco3(z=3)
def func(x, y):
    print('exec')
    return x, y

# zに渡した値は保持されている
func(1, 2)

before exec 3
exec
after exec 3


(1, 2)

デコレータを使うときは、functools.wraps()を使い、実際に実行される関数の名前やDocstringを、もとの関数のものに置き換えるのが一般的

In [9]:
from functools import wraps
def deco4(f):
    # 元の関数を引数にとるデコレーター
    @wraps(f)
    def wrapper(*args, **kwargs):
        print('before exec')
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

In [26]:
@deco4
def func():
    """funcです"""
    print('exec')

print(f'__name__: {func.__name__}')
print(f'__doc__: {func.__doc__}')

__name__: func
__doc__: funcです


functools.wraps()を使わないと、上記の例だとwrapperになり、複数個所で同じデコレータを使うと、同じ関数名が複数存在することになっていしまう。

In [27]:
def deco4(f):
    def wrapper(*args, **kwargs):
        print('before exec')
        v = f(*args, **kwargs)
        print('after exec')
        return v
    return wrapper

In [28]:
@deco4
def func():
    """funcです"""
    print('exec')

print(f'__name__: {func.__name__}')
print(f'__doc__: {func.__doc__}')

__name__: wrapper
__doc__: None


デコレータの実例として、関数の処理時間を計測できるデコレータ

In [17]:
import time

def elapsed_time(f):
    @wraps(f)
    def wrapper(*args, **kwrags):
        start = time.time()
        v = f(*args, **kwrags)
        print(f"{f.__name__}: {time.time() - start}")
        return v
    return wrapper

In [22]:
@elapsed_time
def func(n):
    return sum(i for i in range(n))

print(f'func(1000000)=:{func(1000000)}')
print(f'func(10000000)=:{func(10000000)}')

func: 0.056643009185791016
func(1000000)=:499999500000
func: 0.5320947170257568
func(10000000)=:49999995000000


## functools.lru_cache()  
関数の結果をキャッシュする関数デコレータ。同じ引数での呼び出し結果がすでにキャッシュされてる場合は、  
関数を実行することなくキャッシュ済みの結果を返してくれる。

In [29]:
from functools import lru_cache
from time import sleep

# 最近の呼び出し最大32回分までをキャッシュ
@lru_cache(maxsize=32)
def heavy_function(n):
    sleep(3)
    return n + 1

In [30]:
# 初回なので時間がかかる
heavy_function(2)

3

In [ ]:
# キャッシュにヒットするのですぐに結果を得られる。
heavy_function(2)

## dataclasses.dataclass()

## プロパティ
インスタンスメソッドに@propetyをつけると、そのインスタンスメソッドは()を付けずに呼び出せる。  
setterをつけると、インスタンスメソッドに値の変更があった場合に呼ばれる。

In [1]:
# ECサイトなどで、値引率が価格に反映されることや、値引率に不正な値を設定できないようにするコード
class Book:
    def __init__(self, raw_price):
        if raw_price < 0:
            raise ValueError('price must be positive')
        self.raw_price = raw_price
        self._discounts = 0

    @property
    def discounts(self):
        return self._discounts
    
    @discounts.setter
    def discounts(self, value):
        if value < 0 or 100 < value:
            raise ValueError('discounts must be between 0 and 100')
        self._discounts = value

    @property
    def price(self):
        multi = 100 - self._discounts
        return int(self.raw_price * multi / 100)

In [2]:
book = Book(2000)
print(f'初期の値引率  : {book.discounts}')
print(f'価格        : {book.price}')
# 値引率を設定。値を設定しているため、setterが呼ばれる。
book.discounts = 20
print(f'値引き後の価格: {book.price}')

初期の値引率  : 0
価格        : 2000
値引き後の価格: 1600


## classmethod
### Class.method()で呼び出すことができ、引数clsには、Classが入る。
メリットとしては、  
* 第一引数でクラスが取得できる。  
* クラスの中にあるので、クラスをインポートすれば使える。

In [1]:
# classmethodの場合は、第一引数にclsを使うことが推奨される
class ClassTest:
    class_variable = 0

    @classmethod
    def this_is_class_method(cls):
        cls.class_variable += 1
        return cls.class_variable

ClassTest.this_is_class_method()

1

In [2]:
class Person:
    def __init__(self, first, last):
        self.first = first
        self.last = last

    @classmethod
    def from_str(cls, str):
        first, last = str.split('-')
        # cls(first, last)はPerson(first, last)と同じになり、インスタンスが生成される
        return cls(first, last)

str_1 = "太郎-山田"
str_2 = "一人-殿馬"

person1 = Person.from_str(str_1)
person2 = Person.from_str(str_2)
print(person1.first)
print(person2.last)

太郎
殿馬


## staticmethod
### staticmethodは、毎回同じ結果を出力したいときによく使われる。第一引数を取る必要がない。
### インスタンス変数やインスタンスメソッドにアクセスしないときにも使われる。

In [4]:
class Person:
    def __init__(self, first, last):
        self.first = first
        self.last = last

    # selfなどの第一引数なし
    @staticmethod
    def hello():
        print ("Hello")

# 2つのインスタンスを生成
person1 = Person("太郎", "山田")
person2 = Person("一人", "殿馬")

# person1でもperson2でも出力結果は同じである。
person1.hello()
person2.hello()

Hello
Hello
